In [ ]:
# Some useful modules for notebooks
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdne.conf import Config
import h5py
from tqdne.dataset import extract_sample_from_h5file

config = Config()


In [ ]:
datapath = config.datapath
datapath

In [ ]:
sample_id = 0
with h5py.File(datapath, "r") as f:
    for key in f.keys():
        print(f"Key: {key} -- {f[key].shape}")

    time = f["time_vector"][:]
    waveform, features = extract_sample_from_h5file(f, sample_id)
plt.plot(time, waveform.T)
plt.xlabel("Time [s]")


In [ ]:
signal = waveform[0,:4*1024]
time = time[:4*1024]
plt.plot(time, signal)
plt.xlabel("Time [s]")


In [ ]:
stft_channels = 512
hop_size = 16

In [ ]:
from tifresi.stft import GaussTF, GaussTruncTF
# For faster processin, a truncated window can be used instead
use_truncated_window = True
if use_truncated_window:
    stft_system = GaussTruncTF(hop_size=hop_size, stft_channels=stft_channels)
else:
    stft_system = GaussTF(hop_size=hop_size, stft_channels=stft_channels)



In [ ]:
Y = stft_system.spectrogram(signal, normalize=False)

In [ ]:
np.max(np.abs(Y))

In [ ]:

def log_spectrogram(spectrogram, dynamic_range_dB=60):
    """Compute the log spectrogram representation from a spectrogram."""
    spectrogram = np.abs(spectrogram)  # for safety
    minimum_relative_amplitude = np.max(spectrogram) / 10 ** (dynamic_range_dB / 10)
    return 10 * np.log10(np.clip(spectrogram, a_min=minimum_relative_amplitude, a_max=None))


def inv_log_spectrogram(log_spec):
    """Inverse the log representation of the spectrogram or mel spectrogram."""
    return 10 ** (log_spec / 10)



In [ ]:
# We use the log spectrogram as we have a logarithmic perception of sound energy.
dynamic_range_dB = 80
log_Y= log_spectrogram(Y, dynamic_range_dB)

In [ ]:
plt.figure(dpi=200, figsize=(10,3))
plt.imshow(log_Y)
plt.colorbar()
np.max(log_Y)

In [ ]:
from tifresi.metrics import projection_loss

new_Y = inv_log_spectrogram(log_Y)
print(projection_loss(new_Y, Y))

In [ ]:
new_signal = stft_system.invert_spectrogram(new_Y)


In [ ]:
plt.plot(time, signal, '-b', label="original")
plt.plot(time, new_signal, '-r', label="reconstructed", alpha=0.5)
plt.legend()
plt.xlabel("Time [s]");


In [ ]:
plt.plot(time, signal, '-b', label="original")
plt.plot(time, -new_signal, '-r', label="reconstructed")
plt.legend()
plt.xlabel("Time [s]")
plt.xlim([8, 12]);

In [ ]:
from tifresi.metrics import projection_loss
new_Yp = stft_system.spectrogram(new_signal)
print(projection_loss(new_Y, new_Yp))